# 9. nO CORRER Otros

## 9.1 Explicación KDE

Currently, we have the meaning of each column (variable) that we show here below: 

* client_id: Every client’s unique ID.
* variation: Indicates if a client was part of the experiment.
* visitor_id: A unique ID for each client-device combination.
* visit_id: A unique ID for each web visit/session.
* process_step: Marks each step in the digital process.
* date_time: Timestamp of each web activity.
* clnt_tenure_yr: Represents how long the client has been with Vanguard, measured in years.
* clnt_tenure_mnth: Further breaks down the client’s tenure with Vanguard in months.
* clnt_age: Indicates the age of the client.
* gendr: Specifies the client’s gender.
* num_accts: Denotes the number of accounts the client holds with Vanguard.
* bal: Gives the total balance spread across all accounts for a particular client.
* calls_6_mnth: Records the number of times the client reached out over a call in the past six months.
* logons_6_mnth: The frequency with which the client logged onto Vanguard’s platform over the last six months

## 9.2 Análisis de la demografía de los clientes

In [ ]:
# Análisis de la edad de los clientes
age_distribution = df_demo['clnt_age'].describe()

# Distribución de género
gender_distribution = df_demo['gendr'].value_counts()

# Número de cuentas por cliente
account_distribution = df_demo['num_accts'].value_counts()

age_distribution, gender_distribution, account_distribution

In [ ]:
# Edad promedio
average_age = df_demo['clnt_age'].mean()

# Tenencia promedio
average_tenure = df_demo['clnt_tenure_yr'].mean()

# Agrupar por edad y tenencia
age_tenure_analysis = df_demo.groupby(['clnt_tenure_yr', 'clnt_age']).size().reset_index(name='counts')

average_age, average_tenure, age_tenure_analysis

In [ ]:
# Análisis de saldo en función de la edad
age_balance_correlation = df_demo[['clnt_age', 'bal']].corr()

# Promedio de llamadas y logins por grupo de saldo
call_login_analysis = df_demo.groupby(pd.cut(df_demo['bal'], bins=[0, 100000, 500000, 1000000, 10000000])).agg({'calls_6_mnth': 'mean', 'logons_6_mnth': 'mean'})

age_balance_correlation, call_login_analysis

In [ ]:
correlation_age_accounts = df_demo['clnt_age'].corr(df_demo['num_accts'])
print("Correlación entre Edad y Número de Cuentas:", correlation_age_accounts)

In [ ]:
age_bins = [0, 30, 45, 60, 100]
age_labels = ['Joven', 'Adulto', 'Mayor', 'Anciano']
df_demo['age_group'] = pd.cut(df_demo['clnt_age'], bins=age_bins, labels=age_labels)

avg_accounts_by_age_group = df_demo.groupby('age_group')['num_accts'].mean()
print(avg_accounts_by_age_group)

In [ ]:
demographics_balance = df_demo.groupby(['clnt_age', 'gendr'])['bal'].describe()
demographics_balance

411 Análisis por gender

df_sample_client.groupby(['variation','gendr'])['client_id'].count()

In [ ]:
df1 = df_sample_client.groupby(['variation','gendr'])['client_id'].count()

promedio de llamadas y logins por grupo de saldo

In [ ]:
# Promedio de llamadas y logins por grupo de saldo
call_login_analysis = df_demo.groupby(pd.cut(df_demo['bal'], bins=[0, 100000, 500000, 1000000, 10000000])).agg({'calls_6_mnth': 'mean', 'logons_6_mnth': 'mean'})

call_login_analysis

# 10. (NO CORRER) Contraste de Hipótesis

## 8.1 Prep

In [ ]:
df_conv=pd.read_csv('../sol_data/df_conv.csv')
df_error_counter=pd.read_csv("../sol_data/df_error_counter.csv")

In [ ]:
df_time= pd.read_csv("vanguard - process step diff and step time per client id, per visit_id.csv")

In [ ]:
df_error_counter

,Unnamed: 0,variation,conversion,total,proportion,prop*100
0,0,control,6170.0,138038,0.044698,4.469784
1,1,test,11759.0,175866,0.066863,6.686341


## Z-test proportion test 

Based on https://towardsdatascience.com/a-b-testing-with-binary-data-two-sample-proportions-z-test-5c222d7772ad!

In [ ]:
def create_df_ztest(df,stat):
    pd.DataFrame({
    'variation': ['control', 'test'], 
    'conversion': [
        
        sum(df[df['variation'] == 'Control'].stat),
        sum(df[df['variation'] == 'Test'].stat)
    ], 
  
    'total': [

        len(df[df['variation'] == 'Control']),
        len(df[df['variation'] == 'Test'])],

    'proportion':[
        sum(df[df['variation'] == 'Control'].stat) / len(df[df['variation'] == 'Control']),
        sum(df[df['variation'] == 'Test'].stat) / len(df[df['variation'] == 'Test'])
    ]
})

In [ ]:
# create data frame with counts of conversion in test and control group with extra column with total counts per sample
df_ztest = pd.DataFrame({
    'variation': ['control', 'test'], 
    'conversion': [
        
        sum(df_conv[df_conv['variation'] == 'Control'].conversion),
        sum(df_conv[df_conv['variation'] == 'Test'].conversion)
    ], 
  
    'total': [

        len(df_conv[df_conv['variation'] == 'Control']),
        len(df_conv[df_conv['variation'] == 'Test'])],

    'proportion':[
        sum(df_conv[df_conv['variation'] == 'Control'].conversion) / len(df_conv[df_conv['variation'] == 'Control']),
        sum(df_conv[df_conv['variation'] == 'Test'].conversion) / len(df_conv[df_conv['variation'] == 'Test'])
    ]
})

In [ ]:
df_ztest

,variation,conversion,total,proportion
0,control,14726,23307,0.631827
1,test,18259,26889,0.679051


In [ ]:
df_error_counter

,Unnamed: 0,variation,conversion,total,proportion,prop*100
0,0,control,6170.0,138038,0.044698,4.469784
1,1,test,11759.0,175866,0.066863,6.686341


In [ ]:
#run proportion test
#proportions_ztest(count=[18333, 15201], nobs=[26959,23527], alternative='larger')

In [ ]:
# Z-test definitions: 

# p1 = proportion of successes in sample 1 (control group)
# p2 = proportion of successes in sample 2 (test group)

# n1 = number of trials in sample 1
# n2 = number of trials in sample 2

#The null hypothesis states that there is no difference in the proportions, while 
#the alternative hypothesis says that proportion 1 is less than proportion 2. I.e., there is an effect of the new design.

# H0: p1 = p2
# H1: p1 < p2 (left-tailed test)

In [ ]:
# Estimate sample size for z-test

def z_calc(p1, p2, n1, n2):
    p_star = (p1*n1 + p2*n2) / (n1 + n2)
    return (p2 - p1) / math.sqrt(p_star*(1 - p_star)*((1.0 / n1) + (1.0 / n2)))

def sample_required(p1, p_diff, alpha):
    if p_diff <= 0:  # p_diff = p2-p1
        raise ValueError("p_diff must be > 0")
    n = 1
    while True:
        z = z_calc(p1, p1+p_diff, n1=n, n2=n)
        p = 1 - stats.norm.cdf(z)
        if p < alpha:
            break
        n += 1
    return n

sample_size = sample_required(df_ztest[['proportion'][0]][0],(df_ztest[['proportion'][0]][1] - df_ztest[['proportion'][0]][0]), 0.05)

sample_size

548

The minimum sample size is 1051 for each group. (this is an extra step - in our case the samples were very big anyway)

In [ ]:
# Create new data frame with random sample for the z-test

df_tr = df_conv[df_conv['variation'] == 'Test']
df_ctrl = df_conv[df_conv['variation'] == 'Control']

In [ ]:
df_tr_sample = df_tr.sample(n=20000, random_state=23)
df_ctrl_sample = df_ctrl.sample(n=20000, random_state=23)

NameError: name 'df_tr' is not defined

In [ ]:
df_sample = pd.concat([df_tr_sample, df_ctrl_sample], ignore_index=True)
df_sample.head(2)

,client_id,confirm,start,step_1,step_2,step_3,conversion,completed_&_confused,completed_linear,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,variation,edad_redondeada
0,2342766,1.0,1.0,1.0,1.0,1.0,1,0,1,15,183,63.5,F,2,38319.57,0,3,Test,64
1,8348583,1.0,1.0,1.0,1.0,1.0,1,0,1,6,82,47.5,U,2,52909.43,0,3,Test,48


In [ ]:
df_sample_gr = df_sample.groupby('variation').agg({'conversion':'sum','client_id':'nunique'}).reset_index()

In [ ]:
df_sample_gr['proportion'] = (df_sample_gr['conversion']) / df_sample_gr['client_id']

In [ ]:
df_sample_gr

,variation,conversion,client_id,proportion
0,Control,12635,20000,0.63175
1,Test,13591,20000,0.67955


### Z-test and results

In [ ]:
number_of_successes = [12899, 13626]
total_sample_sizes = [20000, 20000]

In [ ]:
test_stat, p_value = proportions_ztest(number_of_successes, total_sample_sizes, alternative='smaller')

In [ ]:
print("Computed z-test statistic: ", test_stat)
print("Computed p-value: ", p_value)

Computed z-test statistic:  -7.6908142674918025
Computed p-value:  7.310077118377838e-15


 In that case, we see that our result is statistically significant and we have evidence to reject the null hypothesis.

### Proportion z-test vs threshold 105%

In [ ]:
#df_sample_gr

threshold=(sum(df_conv[df_conv['variation'] == 'Control'].conversion) / len(df_conv[df_conv['variation'] == 'Control']))*1.05
threshold

0.6634187154073884

In [ ]:
# run z proportion test of test conversion vs p
# H0: p = p0
# H1: test > p

# Result cannot reject H0


proportions_ztest(count=18333, nobs=26959, value=threshold ,alternative='larger')

(5.8479878733746435, 2.4877740802725258e-09)

### Duration per step testing (PENDIENTE)

In [ ]:
df_time.head()

In [ ]:
df_time.shape

In [ ]:
# boxplot of step_time  
sns.boxplot(  y='step_time', data=df_time)

In [ ]:
# remove nulls from df_time step_time

df_time2 = df_time.dropna(subset=['step_time'])

df_time2['step_time'].describe()

In [ ]:
#running again the boxplot
sns.boxplot(  y='step_time', data=df_time2)

In [ ]:
# Assigning 600 seconds max to the times >= 600s
df_time2['step_time'] = df_time2['step_time'].apply(lambda x : x if x < 600 else 600)


df_time2['step_time'].describe()

In [ ]:
df_time2.head()

In [ ]:
#DO NOT RUN
#removing outliers

def remove_outliers(data, col):
    Q3 = np.quantile(data[col], 0.75)
    Q1 = np.quantile(data[col], 0.25)
    IQR = Q3 - Q1
    
    # Define bounds for the outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    print("Lower Bound:", lower_bound)
    print("Upper Bound:", upper_bound)
    
    # Filter the DataFrame based on the condition
    filtered_data = data[(data[col] > lower_bound) & (data[col] < upper_bound)]

    return filtered_data

# Call the function and print the result
df_time_no_outliers = remove_outliers(df_time2, 'step_time')

In [ ]:
#DO NOT RUN
df_time_no_outliers.shape

In [ ]:
#running again the boxplot, but this time with 'replaced by 600 sec' outliers
sns.boxplot(  y='step_time', data=df_time2)

In [ ]:
# create series with control and test step_time 
time_test = df_time2[df_time2['Variation'] == 'Test']['step_time']
time_control = df_time2[df_time2['Variation'] == 'Control']['step_time']

In [ ]:
time_test.describe() 

In [ ]:
sns.histplot(time_test, kde=True, label='Test')
sns.histplot(time_control, kde=True, label='Control')

# Add legend
plt.legend()

# Show plot
plt.show()

In [ ]:
time_test.describe()

In [ ]:
time_control.describe()

In [ ]:
# mann whitney test for step time
# H0: test <= control
# H1: test > control
stats.mannwhitneyu(time_test, time_control, alternative='greater')

we cannot reject H0, meaning either test is smaller is there's no significant difference in terms of avg time duration

In [ ]:
df_tr_error = df_error_counter[df_error_counter['variation'] == 'Test']
df_ctrl_error = df_error_counter[df_error_counter['variation'] == 'Control']

In [ ]:
df_time2

In [ ]:
df_time2.to_csv('df_time2.csv')

In [ ]:
##Z-test on the error rates

In [ ]:
proportions_ztest(count=[11838, 6358], nobs=[177787,143420], alternative='larger')
#the error count in the test group is significantly larger than the error count on the 
# control group --> which may explain why the process wasn't more efficient and didn't meet
# the required threshold

(27.12298779001062, 2.6376195787310787e-162)